**IMPORT** LIBRARIES

In [105]:
import numpy as np 
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier     #KNN
from sklearn.metrics import accuracy_score

READ THE DATA:

- In this challenge you will use the dataset Titanic disaster.
- With the following lines of code, you can import the dataset form the file "train.csv", extract te column of labels (column "Survived") and read the infos of the other features.

you can also follow kaggle competition:
https://www.kaggle.com/code/amitkumarjaiswal/beginner-s-tutorial-to-titanic-using-scikit-learn/notebook

---



In [114]:
from google.colab import drive
#if you downloaded the file in the Colab directory in your Google Drive, use the following lines of code
drive.mount('/gdrive', force_remount=True)
Tr_file="/gdrive/My Drive/Colab Notebooks/T_db/train.csv" 
Ts_file="/gdrive/My Drive/Colab Notebooks/T_db/test.csv" 
dataset=pd.read_csv(Tr_file)
ts_ds=pd.read_csv(Ts_file)
X_tr=dataset.drop('Survived',axis=1)
X_ts= dataset.drop('Survived',axis=1)

Y_tr=dataset.Survived
Y_ts=dataset.Survived

X_tr.info()
Y_tr

dataset

Mounted at /gdrive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


FEATURES ENGINEERING:

- Evaluate if there are features with missing values and modify them (see https://scikit-learn.org/stable/modules/impute.html)
- Find meaningful features (some features are not needed because do not contain useful infos)
- Encode categorical features with numerical values (see https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features)

**MISSING VALUE WITH A FIXED VALUE** 

In [116]:
# Find missing values in the training set
missing_values = X_tr.isnull().sum()

X_tr['Cabin'].fillna('B96', inplace=True)
X_ts['Cabin'].fillna('B96', inplace=True) 




**MISSING VALUE WITH MEAN VALUE AND MOST FREQUENT VALUE** 

In [117]:
X_tr['Age'].fillna(X_tr['Age'].mean(), inplace=True) 
X_ts['Age'].fillna(X_tr['Age'].mean(), inplace=True) 

X_tr['Embarked'].fillna(X_tr['Embarked'].mode()[0], inplace=True)
X_ts['Embarked'].fillna(X_tr['Embarked'].mode()[0], inplace=True)


**CATEGORICAL FEATURES**

In [118]:

X_tr["Sex"] = pd.Categorical(X_tr["Sex"]); 
X_ts["Sex"] = pd.Categorical(X_ts["Sex"])

X_tr["Sex"] = X_tr["Sex"].cat.codes; 
X_ts["Sex"] = X_ts["Sex"].cat.codes 

X_tr["Embarked"] = pd.Categorical(X_tr["Embarked"]); 
X_ts["Embarked"] = pd.Categorical(X_ts["Embarked"])

X_tr["Embarked"] = X_tr["Embarked"].cat.codes; 
X_ts["Embarked"] = X_ts["Embarked"].cat.codes 

X_tr.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,B96,2
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0
2,3,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,B96,2
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,2
4,5,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,B96,2


*MEANINGFUL FEATURES - SELECTION - EXTRACTION -


In [119]:
# Droping less important feautres
X_tr = X_tr.drop(["PassengerId", "Name", "Fare", "Ticket", "Cabin"], axis=1)
X_ts = X_ts.drop(["PassengerId", "Name", "Fare", "Ticket", "Cabin"], axis=1)
X_ts.head()


,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,1,22.0,1,0,2
1,1,0,38.0,1,0,0
2,3,0,26.0,0,0,2
3,1,0,35.0,1,0,2
4,3,1,35.0,0,0,2


In [120]:
#DO NOT MODIFY THE CLASSIFIER AND THE PERFORMANCE EVALUATION
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_tr,Y_tr)
scores = knn.predict(X_ts)

# Show prediction accuracy
print('\nPrediction accuracy:')
print('{:.2%}\n'.format(accuracy_score(Y_ts, scores)))




Prediction accuracy:
87.43%

